In [1]:
import re
import regex
from collections import Counter
import pandas as pd
import numpy as np
import emoji
import nltk

In [2]:
# import the csv file with all the comments and post togheter
comDB = pd.read_csv(r"com_liwc.csv", sep='\t', engine='python', encoding='utf-8')

# import the csv file with JUST the politicians post
postDB = pd.read_csv(r"postDB.csv", engine='python')

In [3]:
# create the Data Frame
df = pd.DataFrame(data=comDB)
df_post = pd.DataFrame(data=postDB)

# add a new colum with sequence numbers
df['Count']=1
df_post['Count']=1

# print all the DF
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 5)

In [4]:
# comments = df[['c_text']].sample(n=1000, random_state=1).copy() # work with a sample
comments = df[['c_text']].copy()
comments.rename(columns={'c_text':'text'}, inplace=True)

In [5]:
for i in list(np.random.choice(list(comments.index), 5)):
    print(f'Comment {i}')
    print(comments.loc[i]['text'], '\n')

Comment 24533
Intanto i 5 stelle hanno votato la loro candidata presidente del Senato!!!!😂😂😂😂😂😂 

Comment 28325
Evelina Lucantoni non era per  nulla ironico,  tanto che ha affermato  che Salvini  si è  bevuto  il  cervello.. mi pare esplicito.  In quanto a  spiegare ke cose  tanto bene, no, non sono brava . PERÒ  ad oggi,  capisco  ciò  che leggo. 

Comment 46023
Gianni Artisi con un post del genere ti fai solo ridere dietro. 

Comment 12961
Fabrizio Fornasari arroganza perche' vorrei che la legge fosse rispettato e perseguire i
 fascisti??
Credo che lei sia arrogante e fascista.. 

Comment 53495
Acqua pubblica 



In [6]:
print('df shape before dropping row: \t', df.shape)
df = df[df['c_text'].notna()]
print('df shape after dropping row: \t', df.shape)
print('Number of Nan in comments text: ', df['c_text'].isna().sum())

df shape before dropping row: 	 (78175, 244)
df shape after dropping row: 	 (78174, 244)
Number of Nan in comments text:  0


In [7]:
comments = df[['c_text']].copy()
comments.rename(columns={'c_text':'text'}, inplace=True)

In [8]:
comments["text_clean"] = comments["text"].copy()

In [9]:
def lower_casing(text):
    return(text.lower())

In [10]:
def remove_spaces(text):
    return ' '.join([token for token in text.split()])

In [11]:
comments["text_clean"]

0        C'è poco da dire questa è gente che la guerra ...
1                          Marcello fai qualcosa perfavore
                               ...                        
78173    @Utente_Generico @Utente_Generico @matteosalvi...
78174    @matteosalvinimi se dai il taser alle donne ch...
Name: text_clean, Length: 78174, dtype: object

In [12]:
comments["text_clean"] = comments["text_clean"].apply(lambda text: lower_casing(text))



In [13]:
comments

,text,text_clean
0,C'è poco da dire questa è gente che la guerra ...,c'è poco da dire questa è gente che la guerra ...
1,Marcello fai qualcosa perfavore,marcello fai qualcosa perfavore
...,...,...
78173,@Utente_Generico @Utente_Generico @matteosalvi...,@utente_generico @utente_generico @matteosalvi...
78174,@matteosalvinimi se dai il taser alle donne ch...,@matteosalvinimi se dai il taser alle donne ch...


### Removal of Patterns

In [14]:
def remove_patterns(text, patterns):
    for pattern in patterns:
        try: r = re.findall(pattern, text)
        except: r = regex.findall(pattern, text)
        for i in r:
            try: text = re.sub(re.escape(i), ' ', text)
            except: text = regex.sub(regex.escape(i), ' ', text)
    return text

def pattern_freq(docs, pattern):
    p_freq = Counter()
    for text in docs:
        p_found= re.findall(pattern, text)
        for p in p_found:
            p_freq[p] += 1
    return p_freq

In [15]:


PATTERNS = {'urls': re.compile(r'https?://\S+|www\.\S+'),
            'users': re.compile(r'@[\w]*'),
            #'hashtags': re.compile(r'#[\w]*'),
            'emojis': emoji.get_emoji_regexp(),
            'laughs': re.compile(r'\b(?:(?:hah+|ah+|hah+a|ah+a|lo+l) ?)+\b'),
            'blabla': re.compile(r'\b(?:(?:bla+) ?)+\b'),
            'dates': re.compile(r'\b(?:(?:[0-9]+[:\/,\.])+[0-9]+)+\b'),
            'digits': r'#\S+(*SKIP)(*FAIL)|\b\d+\s|\s\d+\s|\s\d+|\b\d+\W|\W\d+\W|\W\d+$' 
            }



In [16]:
comments["text_clean"] = comments["text_clean"].apply(lambda text: remove_patterns(text, PATTERNS.values()))

In [17]:
comments

,text,text_clean
0,C'è poco da dire questa è gente che la guerra ...,c'è poco da dire questa è gente che la guerra ...
1,Marcello fai qualcosa perfavore,marcello fai qualcosa perfavore
...,...,...
78173,@Utente_Generico @Utente_Generico @matteosalvi...,consigliatissima l'africa
78174,@matteosalvinimi se dai il taser alle donne ch...,se dai il taser alle donne che non puoi chia...


In [18]:
import string
punct = string.punctuation
print(f"Punctuation symbols: {punct}")

Punctuation symbols: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [19]:
def remove_punctuation(text, keep=[]):
    punctuation = string.punctuation
    for p in keep:
        punctuation = punctuation.replace(p, "")
    return(''.join([char if not char in punctuation else ' ' for char in text]))

In [20]:
comments['text_clean'] = comments['text_clean'].apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [21]:
comments

,text,text_clean
0,C'è poco da dire questa è gente che la guerra ...,c è poco da dire questa è gente che la guerra ...
1,Marcello fai qualcosa perfavore,marcello fai qualcosa perfavore
...,...,...
78173,@Utente_Generico @Utente_Generico @matteosalvi...,consigliatissima l africa
78174,@matteosalvinimi se dai il taser alle donne ch...,se dai il taser alle donne che non puoi chiama...


In [23]:
from spacy.lang.it.stop_words import STOP_WORDS as it_spacy_stopwords


In [24]:
import nltk
nltk.download('stopwords')
it_nltk_stopwords = nltk.corpus.stopwords.words('italian')



[nltk_data] Downloading package stopwords to /home/diego/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
it_nltk_stopwords

['ad',
 'al',
 'allo',
 'ai',
 'agli',
 'all',
 'agl',
 'alla',
 'alle',
 'con',
 'col',
 'coi',
 'da',
 'dal',
 'dallo',
 'dai',
 'dagli',
 'dall',
 'dagl',
 'dalla',
 'dalle',
 'di',
 'del',
 'dello',
 'dei',
 'degli',
 'dell',
 'degl',
 'della',
 'delle',
 'in',
 'nel',
 'nello',
 'nei',
 'negli',
 'nell',
 'negl',
 'nella',
 'nelle',
 'su',
 'sul',
 'sullo',
 'sui',
 'sugli',
 'sull',
 'sugl',
 'sulla',
 'sulle',
 'per',
 'tra',
 'contro',
 'io',
 'tu',
 'lui',
 'lei',
 'noi',
 'voi',
 'loro',
 'mio',
 'mia',
 'miei',
 'mie',
 'tuo',
 'tua',
 'tuoi',
 'tue',
 'suo',
 'sua',
 'suoi',
 'sue',
 'nostro',
 'nostra',
 'nostri',
 'nostre',
 'vostro',
 'vostra',
 'vostri',
 'vostre',
 'mi',
 'ti',
 'ci',
 'vi',
 'lo',
 'la',
 'li',
 'le',
 'gli',
 'ne',
 'il',
 'un',
 'uno',
 'una',
 'ma',
 'ed',
 'se',
 'perché',
 'anche',
 'come',
 'dov',
 'dove',
 'che',
 'chi',
 'cui',
 'non',
 'più',
 'quale',
 'quanto',
 'quanti',
 'quanta',
 'quante',
 'quello',
 'quelli',
 'quella',
 'quelle',
 'q

In [26]:
it_stop_words=[
     'a',
     'adesso',
     'ai',
     'al',
     'alla',
     'allo',
     'allora',
     'almeno',
     'altra',
     'altre',
     'altri',
     'altro',
     'anche',
     'ancora',
     'appena',
     'appieno',
     'avanti',
     'aver',
     'avere',
     'aveva',
     'avevano',
     'basta',
     'bastare',
     'bisogna',
     'bisognare',
     'bo',
     'boh',
     'capire',
     'capisco',
     'capite',
     'capito',
     'c',
     "c'",
     'ce',
     'certo',
     'che',
     'chi',
     'cio',
     'ciò',
     'cm',
     'cmq',
     'cn',
     'come',
     'comunque',
     'con',
     'cosa',
     'cose',
     'cosi',
     'così',
     'credo',
     'cs',
     'cui',
     'd',
     "d'",
     'da',
     'dà',
     'dá',
     'dapprima',
     'data',
     'date',
     'dato',
     'davvero',
     'de',
     'del',
     'dell',
     'della',
     'dello',
     'dentro',
     'detto',
     'deve',
     'devo',
     'devono',
     'di',
     'diciamo',
     'dico',
     'dicono',
     'dite',
     'dobbiamo',
     'dovere',
     'dovete',
     'dovrebbe',
     'dovrebbero',
     'dovresti',
     'doppio',
     'due',
     'e',
     'è',
     'ecco',
     'eccoli',
     'eccolo',
     'er',
     'essere',
     'fare',
     'finalmente',
     'finche',
     'finchè',
     'fino',
     'forse',
     'fra',
     'giu',
     'giusto',
     'guarda',
     'ha',
     'hai',
     'hanno',
     'ho',
     'i',
     'il',
     'immediatamente',
     'infatti',
     'invece',
     'io',
     'k',
     'ke',
     'l',
     "l'",
     'la',
     'le',
     'lei',
     'letto',
     'lo',
     'loro',
     'lui',
     'lungo',
     'ma',
     'magari',
     'man',
     'mano',
     'me',
     'meglio',
     'mentre',
     'mo',
     'modo',
     'molta',
     'molti',
     'molto',
     'neanche',
     'nei',
     'nella',
     'nemmeno',
     'no',
     'noi',
     'nome',
     'non',
     'nn',
     'nostro',
     'notte',
     'nove',
     'nulla',
     'nuova',
     'nuove',
     'nuovi',
     'nuovo',
     'o',
     'oggi',
     'ogni',
     'ok',
     'okay',
     'okey',
     'oltre',
     'ora',
     'oramai',
     'ormai',
     'otto',
     'pare',
     'pensa',
     'pensi',
     'penso',
     'perche',
     'perchè',
     'perché',
     'pero',
     'però',
     'peró',
     'piu',
     'più',
     'piú',
     'poco',
     'possiamo',
     'possibile',
     'posso',
     'possono',
     'posto',
     'prescindere',
     'prima',
     'primo',
     'proprio',
     'ps',
     'puoi',
     'purtroppo',
     'qua',
     'qualche',
     'qualcosa',
     'qualcuno',
    'qualunque',
    'qualsiasi',
     'quarto',
     'quasi',
     'quattro',
     'quel',
     'quello',
     'quest',
     'questa',
     'queste',
     'questi',
     'questo',
     'qua',
     'qui',
     'quindi',
     'quinto',
     'sa',
     'sara',
     'scusa',
     'scusate',
     'secondo',
     'sei',
     'sembra',
     'sembrava',
     'senso',
     'senza',
     'sette',
     'sia',
     'siamo',
     'siete',
     'solo',
     'sono',
     'sopra',
     'soprattutto',
     'sotto',
     'speriamo',
     'spero',
     'stanotte',
     'stasera',
     'stati',
     'stata',
     'stato',
     'stessa',
     'stesso',
     'sta',
     'ste',
     'sti',
     'sto',
     'su',
     'subito',
     'sul',
     'sulla',
     'tanta',
     'tante',
     'tanti',
     'tanto',
     'te',
     'terzo',
     'tipa',
     'tipe',
     'tipi',
     'tipo',
     'tra',
     'tramite',
     'tre',
     'triplo',
     'troppa',
     'troppe',
     'troppi',
     'troppo',
     'tutta',
     'tutte',
     'tutti',
     'tutto',
     'u',
     'ultima',
     'ultimo',
     'un',
     'una',
     'uno',
     'va',
     'vabbe',
     'vabbè',
     'vai',
     'vanno',
     'vedere',
     'vedo',
     'vengo',
     'venire',
     'venuta',
     'venute',
     'venuti',
     'venuto',
     'vera',
     'veramente',
     'vere',
     'veri',
     'vero',
     'verso',
     'viene',
     'vista',
     'viste',
     'visti',
     'visto',
     'voi',
     'vogliamo',
     'voglio',
     'vogliono',
     'volta',
     'volte',
     'vorrei',
     'vorreste',
     'vorresti',
     'vorrebbero',
     'vostro',
     'vuoi',
     'vuol',
     'vuole',
     'x',
     'xche',
     'xché',
     'xk',
     'xke',
     'xké']

In [27]:
set_stop=set(it_stop_words)

In [28]:
allstop=set(it_nltk_stopwords)|set_stop

In [29]:
def remove_stop(text):
    return(' '.join([word for word in text.split() if word not in allstop]))

In [30]:
def replace_upbra(text):
    return re.sub('“',' ',text)

In [31]:
def replace_tilde(text):
    return re.sub('’',' ',text)

In [32]:
comments['text_clean']=comments['text_clean'].apply(lambda text: remove_stop(text))
comments['text_clean']=comments['text_clean'].apply(lambda text: replace_upbra(text))
comments['text_clean']=comments['text_clean'].apply(lambda text: replace_tilde(text))

In [34]:
comments['text_clean'] = comments['text_clean'].apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [35]:
comments['text_clean']=comments['text_clean'].apply(lambda text: remove_stop(text))


In [37]:
comments['text_clean'] = comments['text_clean'].apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [38]:
# multiple vowels? multiple consonants?
import re
def remove_duplicated(text):
    return re.sub("(.)\\1{2,}", "\\1", text)



In [39]:
comments['text_clean']=comments['text_clean'].apply(lambda text: remove_duplicated(text))

## Remove not significant one-char, two-char, three-char

### 1-char non significant words

In [41]:
word3=[]
for comment in comments['text_clean']:
    a=' '.join([word for word in comment.split() if len(word)<2])
    for word in a:
        word3.append(word)
        
    

In [42]:
one_word_set=set(word3)

In [43]:
one_word_set

{' ',
 '#',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 'a',
 'b',
 'c',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '¡',
 '«',
 '\xad',
 '¯',
 '°',
 '·',
 'º',
 '»',
 '×',
 'à',
 'é',
 'ê',
 'ë',
 'ì',
 'í',
 'î',
 'ù',
 'ė',
 'ĺ',
 'ǝ',
 '\u200b',
 '–',
 '—',
 '‘',
 '”',
 '„',
 '•',
 '…',
 '\u202a',
 '\u202c',
 '›',
 '\u2066',
 '\u2069',
 '€',
 '℅',
 '⇦',
 '⇨',
 '⋅',
 '★',
 '☆',
 '♡',
 '⚘',
 '⛤',
 '《',
 '》',
 'ツ',
 '️',
 '￼',
 '𝗔',
 '𝗘',
 '🏻'}

In [44]:
def remove_ones(text):
    return(' '.join([word for word in text.split() if word not in one_word_set]))

In [45]:
comments['text_clean']=comments['text_clean'].apply(lambda text: remove_ones(text))

In [47]:
''' CHECK '''
word3=[]
for comment in comments['text_clean']:
    a=' '.join([word for word in comment.split() if len(word)<2])
    for word in a:
        word3.append(word)
        

In [48]:
word3

[]

### 2-char non significant words

In [49]:
word4=[]

for comment in comments['text_clean']:
    a=[word for word in comment.split() if len(word)==2]
    if a:
        if len(a)>1:
            for word in a:
                word4.append(word)
        else:
            b=''.join(a)
            word4.append(b)

In [50]:
two_words_set=set(word4)

In [51]:
two_words_set

{'#a',
 '#l',
 '#m',
 '#x',
 '00',
 '0€',
 '18',
 '3o',
 '50',
 '53',
 '5s',
 '80',
 '99',
 'a4',
 'a7',
 'aa',
 'ab',
 'ac',
 'ae',
 'af',
 'ag',
 'ah',
 'aj',
 'ak',
 'al',
 'am',
 'an',
 'ao',
 'ap',
 'aq',
 'ar',
 'as',
 'at',
 'au',
 'av',
 'az',
 'aà',
 'a€',
 'ba',
 'bb',
 'bc',
 'be',
 'bi',
 'bl',
 'bn',
 'bo',
 'bp',
 'br',
 'bs',
 'bt',
 'bu',
 'by',
 'bz',
 'bè',
 'bé',
 'ca',
 'cb',
 'cc',
 'cd',
 'ch',
 'cl',
 'co',
 'cp',
 'ct',
 'cu',
 'cv',
 'cz',
 'cà',
 'cè',
 'cì',
 'db',
 'dc',
 'de',
 'dj',
 'dk',
 'dl',
 'dm',
 'dn',
 'do',
 'dp',
 'dr',
 'ds',
 'dt',
 'du',
 'dv',
 'dx',
 'dé',
 'dì',
 'dí',
 'dò',
 'dú',
 'ea',
 'ec',
 'ee',
 'eh',
 'ei',
 'el',
 'en',
 'eo',
 'es',
 'et',
 'eu',
 'ev',
 'ex',
 'è',
 'e…',
 'f1',
 'f4',
 'f5',
 'f6',
 'fa',
 'fb',
 'fc',
 'fe',
 'ff',
 'fg',
 'fi',
 'fn',
 'fo',
 'fq',
 'fr',
 'fs',
 'ft',
 'fv',
 'fw',
 'fà',
 'fá',
 'fù',
 'g7',
 'g8',
 'ga',
 'gb',
 'gf',
 'gg',
 'gi',
 'gl',
 'gm',
 'go',
 'gr',
 'gt',
 'gu',
 'gz',
 'ha',

In [52]:
# manualli select relevant acronyms?
# to keep: pd, ok
two_words_set=set(word4)
two_words_set.remove('pd')
two_words_set.remove('ok')

In [53]:
def remove_two(text):
    return(' '.join([word for word in text.split() if word not in two_words_set]))

In [54]:
comments['text_clean']=comments['text_clean'].apply(lambda text: remove_two(text))

In [55]:
'''CHECK'''
word4=[]

for comment in comments['text_clean']:
    a=[word for word in comment.split() if len(word)==2]
    if a:
        if len(a)>1:
            for word in a:
                word4.append(word)
        else:
            b=''.join(a)
            word4.append(b)

In [56]:
set(word4)

{'ok', 'pd'}

### 3-char non significant words

In [57]:
word5=[]

for comment in comments['text_clean']:
    a=[word for word in comment.split() if len(word)==3]
    if a:
        if len(a)>1:
            for word in a:
                word5.append(word)
        else:
            b=''.join(a)
            word5.append(b)

In [58]:
three_words_set=set(word5)

In [59]:
three_words_set

{'bar',
 'bin',
 'max',
 'tte',
 'dna',
 'loi',
 'tti',
 'n9n',
 'pi0',
 'età',
 'bhe',
 'fit',
 'ldl',
 'sga',
 'tdc',
 'ció',
 'dri',
 'al9',
 'eso',
 'tlc',
 'bee',
 'giä',
 'ené',
 'gia',
 '𝗗𝗘𝗜',
 'dla',
 'in#',
 'zio',
 'lud',
 '«la',
 'gum',
 'nyc',
 'deb',
 'une',
 'peg',
 '€ed',
 'tvb',
 'bon',
 'luo',
 'noo',
 'acc',
 'ted',
 'gnè',
 'toh',
 'pun',
 'ztl',
 'ugl',
 'kgb',
 '‘il',
 'mii',
 'bes',
 'for',
 'sit',
 'wie',
 'tem',
 'drl',
 'mai',
 'dir',
 'noè',
 'tso',
 'bro',
 'unp',
 '25o',
 'vax',
 '10a',
 'pvt',
 'amp',
 'stx',
 'par',
 'yep',
 'msi',
 'pay',
 'chw',
 'ars',
 'boy',
 'qlo',
 'llo',
 'eco',
 'see',
 'bue',
 'omg',
 'sie',
 'tiè',
 'avv',
 'dic',
 'den',
 'von',
 'off',
 'pps',
 'ncc',
 'wir',
 '#fi',
 'use',
 'vlt',
 'tes',
 'enf',
 'smi',
 'iti',
 'nmd',
 'mmh',
 'deo',
 'car',
 'wla',
 'opa',
 'gnl',
 'put',
 'sii',
 'vie',
 'his',
 'ass',
 'mou',
 'old',
 'stå',
 'qmq',
 'vol',
 'str',
 '‘di',
 'puo',
 'one',
 'lvi',
 'asi',
 'maa',
 'ivo',
 'cit',
 'gru',


In [60]:
three_words_set.remove('udc')
three_words_set.remove('rom')
three_words_set.remove('fmi')
three_words_set.remove('via')
three_words_set.remove('onu')
three_words_set.remove('dio')
three_words_set.remove('pdl')
three_words_set.remove('tg2')
three_words_set.remove('caz')
three_words_set.remove('fdi')
three_words_set.remove('tg5')
three_words_set.remove('vip')
three_words_set.remove('zio')
three_words_set.remove('czz')
three_words_set.remove('psi')
three_words_set.remove('gdp')
three_words_set.remove('ano')
three_words_set.remove('m5s')
three_words_set.remove('età')
three_words_set.remove('bce')
three_words_set.remove('gay')
three_words_set.remove('tg7')
three_words_set.remove('tvb')
three_words_set.remove('phd')
three_words_set.remove('gap')
three_words_set.remove('mld')
three_words_set.remove('ogm')

In [61]:
def remove_three(text):
    return(' '.join([word for word in text.split() if word not in three_words_set]))

In [62]:
comments['text_clean']=comments['text_clean'].apply(lambda text: remove_three(text))

In [63]:
''' CHECK '''
word5=[]

for comment in comments['text_clean']:
    a=[word for word in comment.split() if len(word)==3]
    if a:
        if len(a)>1:
            for word in a:
                word5.append(word)
        else:
            b=''.join(a)
            word5.append(b)

In [64]:
set(word5)

{'ano',
 'bce',
 'caz',
 'czz',
 'dio',
 'età',
 'fdi',
 'fmi',
 'gap',
 'gay',
 'gdp',
 'm5s',
 'mld',
 'ogm',
 'onu',
 'pdl',
 'phd',
 'psi',
 'rom',
 'tg2',
 'tg5',
 'tg7',
 'tvb',
 'udc',
 'via',
 'vip',
 'zio'}

### LONG WORDS 

In [68]:
# spaces and puctuation double check
comments['text_clean'] = comments['text_clean'].apply(lambda text: remove_punctuation(text, keep="#")).apply(lambda text: remove_spaces(text))

In [69]:
comments

,text,text_clean
0,C'è poco da dire questa è gente che la guerra ...,dire gente guerra portata casa rimpatriati
1,Marcello fai qualcosa perfavore,marcello perfavore
...,...,...
78173,@Utente_Generico @Utente_Generico @matteosalvi...,consigliatissima africa
78174,@matteosalvinimi se dai il taser alle donne ch...,taser donne chiamare pagliacci tali vedrai eff...


In [70]:
for i in list(np.random.choice(list(comments.index), 10)):
    print(f'Comment {i}')
    print(comments.loc[i]['text'])
    print(comments.loc[i]['text_clean'], '\n')
    print()

Comment 54858
Purtroppo in questa Italia così imprevedibile, si deve combattere per l’onesta’ come se fosse un valore aggiunto... Auguri e... avanti tuttaaaa
italia imprevedibile combattere onesta valore aggiunto auguri tutta 


Comment 70072
Tajany voglio solo dirle a lei tutto il centrodestra che :
L'italia va male e non c'è lavoro e tutto vero.
Però non diamo sempre colpa al governo del periodo
Perché quando c'eravate voi 
E voi avete governato per almeno 10 anni.
Non he che si andasse avanti tanto meglio.
Per qui non facciamo sempre di tutta un erba un fascio.
Idem quando c'èra la sinistra ?
Grazie 
Lasciamoli lavorare poi alla fine di tutto saranno poi i cittadini italiani che giudicheranno sé avranno fatto bene o meno.
Grazie
tajany dirle centrodestra italia male lavoro diamo sempre colpa governo periodo quando governato anni andasse sempre erba fascio idem quando sinistra grazie lasciamoli lavorare fine cittadini italiani giudicheranno fatto bene meno grazie 


Comment 48741
Rob

In [71]:
def sanity_check(text):
    return [char for char in text if char not in 'abcdefghijklmnopqrstuwvzxy# ']

In [72]:
spec_char=[]
for i in comments['text_clean'].index:
    a=sanity_check(comments['text_clean'][i])
    if a:
        if len(a)>1:
            for char in a:
                spec_char.append(char)
        else:
            b=''.join(a)
            spec_char.append(b)

In [73]:
set(spec_char)

{'0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '¡',
 '§',
 '«',
 '°',
 '´',
 '¶',
 '»',
 '¿',
 'à',
 'á',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ì',
 'í',
 'î',
 'ï',
 'ð',
 'ñ',
 'ò',
 'ó',
 'ô',
 'ö',
 'ø',
 'ù',
 'ú',
 'ü',
 'ď',
 'ģ',
 'ı',
 'ĺ',
 'ľ',
 'ł',
 'ť',
 'ź',
 'ǝ',
 'ț',
 'ɐ',
 'ɔ',
 'ɯ',
 'ɹ',
 'ʇ',
 'ʌ',
 '̀',
 '̷',
 'έ',
 'α',
 'η',
 'κ',
 'λ',
 'μ',
 'ρ',
 'ω',
 'а',
 'е',
 'и',
 'к',
 'н',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ы',
 'ן',
 'أ',
 'ا',
 'ح',
 'ض',
 'ل',
 'ى',
 'ᴉ',
 '\u200b',
 '–',
 '—',
 '‘',
 '”',
 '„',
 '…',
 '\u202a',
 '\u202c',
 '⁰',
 '€',
 'ⅎ',
 '☆',
 '♡',
 '♤',
 '⚘',
 '⛦',
 '《',
 '》',
 '・',
 '️',
 '？',
 '｀',
 '￼',
 '�',
 '𝐀',
 '𝐈',
 '𝐏',
 '𝐑',
 '𝐕',
 '𝐚',
 '𝐜',
 '𝐝',
 '𝐞',
 '𝐡',
 '𝐢',
 '𝐦',
 '𝐧',
 '𝐨',
 '𝐩',
 '𝐫',
 '𝐬',
 '𝐭',
 '𝗔',
 '𝗕',
 '𝗖',
 '𝗗',
 '𝗘',
 '𝗙',
 '𝗚',
 '𝗜',
 '𝗠',
 '𝗡',
 '𝗢',
 '𝗣',
 '𝗤',
 '𝗥',
 '𝗦',
 '𝗧',
 '𝗨',
 '𝗩',
 '𝗭',
 '𝙂',
 '𝙖',
 '𝙙',
 '𝙞',
 '𝙣',
 '𝙤',
 '𝙧',
 '𝙨'}

In [79]:
def multiple_replace(dicti, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dicti.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dicti[mo.string[mo.start():mo.end()]], text) 

dicti = {
    'à': "a",
    'á' : "a",
    'â' : "a",
    'è' : 'e',
    'é' :'e',
    'ê' :'e',
    'ì' :'i',
    'í' :'i',
    'î' :'i',
    'ï' :'i',
    'ò' : 'o',
    'ó' : 'o',
    'ô' :'o',
    'ö' : 'o',
    'ø' : 'o',
    'ù' :'u',
    'ú' :'u',
    'ü' : 'u',
    'ď' : 'd',
    'ģ' : 'g',
    'ĺ' : 'l',
    'ľ' : 'l',
    'ł' : 'l',
    'ť' : 't',
    'ź' : 'z',
    '𝐀' : 'a',
    '𝐈' : 'i',
    '𝐏' : 'p',
    '𝐑' : 'r',
    '𝐕' : 'v',
    '𝐚' : 'a',
    '𝐜' : 'c',
    '𝐝' : 'd',
    '𝐞' : 'e',
    '𝐡' : 'h',
    '𝐢' : 'i',
    '𝐦' : 'm',
    '𝐧' : 'n',
    '𝐨' : 'o',
    '𝐩' : 'p',
    '𝐫' : 'r',
    '𝐬' : 's',
    '𝐭' : 't',
    '𝗔':'a',
     '𝗕': 'b',
     '𝗖': 'c',
     '𝗗': 'd',
     '𝗘': 'e',
     '𝗙': 'f',
     '𝗚': 'g',
     '𝗜': 'i',
     '𝗠': 'm',
     '𝗡': 'n',
     '𝗢': 'o',
     '𝗣': 'p',
     '𝗤': 'q',
     '𝗥': 'r',
     '𝗦': 's',
     '𝗧': 't',
     '𝗨': 'u',
     '𝗩': 'v',
     '𝗭': 'z',
     '𝙂': 'g',
     '𝙖': 'a',
     '𝙙': 'd',
     '𝙞': 'i',
     '𝙣': 'n',
     '𝙤': 'o',
     '𝙧': 'r',
     '𝙨': 's'
  } 


In [80]:
comments['text_clean']=comments['text_clean'].apply(lambda text: multiple_replace(dicti,text))

In [83]:
spec_char=[]
for i in comments['text_clean'].index:
    a=sanity_check(comments['text_clean'][i])
    if a:
        if len(a)>1:
            for char in a:
                spec_char.append(char)
        else:
            b=''.join(a)
            spec_char.append(b)
set_spec_char=set(spec_char)

In [84]:
def remove_spec_char(text):
    return(''.join([char for char in text if char not in set_spec_char]))

In [85]:
comments['text_clean']=comments['text_clean'].apply(lambda text: remove_spec_char(text))

In [88]:
''' CHECK FOR SPECIAL CHAR'''
spec_char=[]
for i in comments['text_clean'].index:
    a=sanity_check(comments['text_clean'][i])
    if a:
        if len(a)>1:
            for char in a:
                spec_char.append(char)
        else:
            b=''.join(a)
            spec_char.append(b)
set_spec_char=set(spec_char)

In [89]:
set_spec_char

set()

In [122]:
final=comments[comments['text_clean']!='']['text_clean']

In [99]:
final.to_csv('/home/diego/Desktop/Networks/text_cleaned_final.csv')

### Check for Duplicates Comments

In [93]:
index_null_comments=[]
for i in final.index:
    if not final[i]:
        index_null_comments.append(i)
    

In [94]:
len(index_null_comments)

2399

In [95]:
a=final[final.duplicated()]

In [96]:
list_dup=[]
for dup in a:
    if dup:
        list_dup.append(dup)

In [97]:
len(list_dup)

13550

In [98]:
len(set(list_dup))

7072

In [100]:
text=comments['text']

In [101]:
text.duplicated(False)

0        False
1        False
         ...  
78173    False
78174    False
Name: text, Length: 78174, dtype: bool

In [102]:
text[text.duplicated(False)].value_counts()

Auguri                                        227
Buona Pasqua                                   86
                                             ... 
Bisogna tagliarli testa e gambe  subito...      2
Anche a te e famiglia                           2
Name: text, Length: 6727, dtype: int64

In [103]:
#extract duplicated comments
#
dup_comments=text[text.duplicated()]

In [104]:
dup_comments

27       Castrazione chirurgica quella chimica è una bu...
248      X carità cara Mara... Non vedo l'ora che arriv...
                               ...                        
78160    @Utente_Generico @matteosalvinimi @Utente_Gene...
78161    @matteosalvinimi Stai facendo soldi senza aver...
Name: text, Length: 8475, dtype: object

In [105]:
len(dup_comments)

8475

In [106]:
dup_comments.index

Int64Index([   27,   248,   250,   261,   398,   399,   687,   698,   701,
              727,
            ...
            78148, 78149, 78150, 78151, 78152, 78155, 78156, 78158, 78160,
            78161],
           dtype='int64', length=8475)

In [107]:
a=comDB.loc[dup_comments.index].p_PARTITO.value_counts()

In [108]:
a=comDB.loc[dup_comments.index]

In [109]:
len(a['Origin_file_order'].unique())

8475

In [110]:
a

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP,Count
27,94285,FB,667483386685540_1835929416507592,2019-04-29T15:12:05Z,SILVIA SERAFINA DETTA NARDONE SARDONE,F,EFDD,LEGA,Lega,governo,destra,no,I vermi protagonisti delle ultime violenze sul...,2335,571,445,13,113152,0,negativo,None,Other,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,Castrazione chirurgica quella chimica è una bu...,1,silviasardone,I vermi protagonisti delle ultime violenze sul...,problematico,probl-hate,incivile,negativo,Altro,NaN,Other,Other,24,12.0,33.33,45.83,4.17,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8.33,8.33,0.0,8.33,4.17,4.17,0.0,4.17,0.0,4.17,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,4.17,0.0,0.0,0.0,0.00,4.17,12.50,0.0,0.00,0.0,0.0,4.17,0.00,4.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17,0.0,0.0,0.0,0.00,0.0,0,0.0,0.00,0,0.0,0.0,8.33,0.00,0.0,0.0,8.33,8.33,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,203,40.6,40.89,31.03,2.46,0.0,0.99,0.0,0.0,0.0,0.99,0.0,7.88,10.84,0.00,1.48,1.48,0.0,0.49,0.00,0.0,0.00,0.0,2.46,0.49,0.49,0.00,0.49,0.0,0.00,0.49,0.00,0.0,0.49,1.97,0.49,1.48,0.0,0.0,0.0,0.49,0.49,0.99,0.49,0.49,0.0,0.0,0.99,1.97,0.49,0.99,0.0,0.0,0.49,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.49,0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.49,0.0,0,0.0,0.49,0,0.49,0.0,0.00,0.0,0.0,0.49,16.75,1.48,8.37,0.49,0.0,0.0,0.99,5.42,0.00,0.00,0.0,0.00,1
248,93386,FB,206708532746370_2331895640227638,2019-04-29T10:28:37Z,MARA BIZZOTTO,F,EFDD,LEGA,Lega,governo,destra,ne,❌❌ C’è chi sta con gli IMMIGRATI CLANDESTINI e...,277,67,182,129,4739,0,problematico,Rifugiati,rifugiati,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,X carità cara Mara... N

In [111]:
pd.set_option('display.max_rows', 111)

In [112]:
print(a)

       Origin_file_order     Site                              p_id  \
27                 94285       FB  667483386685540_1835929416507592   
248                93386       FB  206708532746370_2331895640227638   
250                93408       FB  206708532746370_2331895640227638   
261                31594       FB  206708532746370_2321902121226990   
398                90236       FB  667483386685540_1835929416507592   
...                  ...      ...                               ...   
78155              98012  Twitter             1.12350320635567E+018   
78156              90650  Twitter             1.12324220237583E+018   
78158              96194  Twitter             1.12317637480466E+018   
78160              95777  Twitter             1.12276187018688E+018   
78161              90446  Twitter             1.12185694620055E+018   

                dateCreated                           p_politician p_gender  \
27     2019-04-29T15:12:05Z  SILVIA SERAFINA DETTA NARDONE SARDONE  

In [113]:
pd.set_option('display.max_rows', 111)

In [114]:
print(comDB.loc[dup_comments.index].c_replyToUser.value_counts())

Anonymous                           1620
fittoraffaele                        267
giorgiameloni.paginaufficiale        264
marabizzotto                         240
salviniofficial                      229
CarloCalenda                         214
matteosalvinimi                      205
LuigiDiMaio                          204
BrignoneBea                          193
ienadinogiarrusso                    182
SilvioBerlusconi                     159
alessandra.cappellari.mantova        157
DSantanche                           156
silviasardone                        146
Ale_Mussolini_                       140
Susannaceccardi87                    137
pinapic                              132
cattaneodante                        132
simona.bonafe.5                      130
CaioM2019                            130
lara.comi.official                   127
civati                               123
danielaaiuto75                       122
GiorgiaMeloni                        115
pagina.DavidSass

In [115]:
comDB.loc[dup_comments.index]

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,p_dx_sx,p_CIRCOSCRIZIONE,p_text,p_favoriteCount,p_shareCount,p_replyCount,p_replyEval,p_numComments,p_numFakeTags,p_rating,p_category,p_topic,p_campagna,p_camapagna2,Target1,Target2,p_targe1-2,target1_s-p,target1_pol,c_text,c_level,c_replyToUser,c_replyToText,c_rating,c_rating3,c_ratingCivile,c_ratingPosNeg,c_category,Unnamed: 38,c_topic,isPersonal,c_WC,c_WPS,c_Sixltr,c_Dic,c_pronomi,c_Io,c_Noi,c_Se,c_Tu,c_Altri,c_Negazio,c_Consen,c_Articol,c_Prepos,c_Numero,c_Affett,c_Sen_Pos,c_Emo_Pos,c_Ottimis,c_Emo_Neg,c_Ansia,c_Rabbia,c_Tristez,c_Mec_Cog,c_Causa,c_Intros,c_Discrep,c_Inibiz,c_possib,c_Certez,c_Proc_Sen,c_Vista,c_Udito,c_Sentim,c_Social,c_Comm,c_Rif_gen,c_amici,c_Famigl,c_Umano,c_Tempo,c_Passato,c_Present,c_Futuro,c_Spazio,c_Sopra,c_Sotto,c_Inclusi,c_Esclusi,c_Movimen,c_Occupaz,c_Scuola,c_Lavoro,c_Raggiun,c_Svago,c_Casa,c_Sport,c_TV_it,c_Musica,c_Soldi,c_Metafis,c_religio,c_Morte,c_Fisico,c_Corpo,c_Sesso,c_Mangiare,c_Dormire,c_Cura_cor,c_parolac,c_Non_flu,c_riempiti,c_Voi,c_Lui_lei,c_Loro,c_Condizio,c_Transiti,c_P_pass,c_gerundio,c_Passivo,c_Essere,c_Avere,c_Formale,c_Io_Ver,c_Tu_Verbo,c_Lui_Verb,c_Noi_Verb,c_Voi_Verb,c_Loro_Ver,c_AllPunc,c_Period,c_Comma,c_Colon,c_SemiC,c_Qmark,c_Exclam,c_Dash,c_Quote,c_Apostro,c_Parenth,c_OtherP,p_WC,p_WPS,p_Sixltr,p_Dic,p_pronomi,p_Io,p_Noi,p_Se,p_Tu,p_Altri,p_Negazio,p_Consen,p_Articol,p_Prepos,p_Numero,p_Affett,p_Sen_Pos,p_Emo_Pos,p_Ottimis,p_Emo_Neg,p_Ansia,p_Rabbia,p_Tristez,p_Mec_Cog,p_Causa,Ip_ntros,p_Discrep,Ip_nibiz,p_possib,p_Certez,p_Proc_Sen,p_Vista,p_Udito,p_Sentim,p_Social,p_Comm,p_Rif_gen,p_amici,p_Famigl,p_Umano,p_Tempo,p_Passato,p_Present,p_Futuro,p_Spazio,p_Sopra,p_Sotto,Ip_nclusi,p_Esclusi,p_Movimen,p_Occupaz,p_Scuola,p_Lavoro,p_Raggiun,p_Svago,p_Casa,p_Sport,p_TV_it,p_Musica,p_Soldi,p_Metafis,p_religio,p_Morte,p_Fisico,p_Corpo,p_Sesso,p_Mangiare,p_Dormire,p_Cura_cor,p_parolac,p_Non_flu,p_riempiti,p_Voi,p_Lui_lei,p_Loro,p_Condizio,p_Transiti,p_P_pass,p_gerundio,p_Passivo,p_Essere,p_Avere,p_Formale,p_Io_Ver,p_Tu_Verbo,p_Lui_Verb,p_Noi_Verb,p_Voi_Verb,p_Loro_Ver,p_AllPunc,p_Period,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP,Count
27,94285,FB,667483386685540_1835929416507592,2019-04-29T15:12:05Z,SILVIA SERAFINA DETTA NARDONE SARDONE,F,EFDD,LEGA,Lega,governo,destra,no,I vermi protagonisti delle ultime violenze sul...,2335,571,445,13,113152,0,negativo,None,Other,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non politico,Castrazione chirurgica quella chimica è una bu...,1,silviasardone,I vermi protagonisti delle ultime violenze sul...,problematico,probl-hate,incivile,negativo,Altro,NaN,Other,Other,24,12.0,33.33,45.83,4.17,0.0,0.0,0.0,0.0,0.0,0.00,0.0,8.33,8.33,0.0,8.33,4.17,4.17,0.0,4.17,0.0,4.17,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,4.17,0.0,0.0,0.00,0.00,4.17,12.50,0.00,0.00,0.0,0.0,4.17,0.00,4.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.17,0.0,0.00,0.0,0.00,0.0,0,0.00,0.00,0,0.00,0.0,8.33,0.00,0.0,0.00,8.33,8.33,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,203,40.60,40.89,31.03,2.46,0.00,0.99,0.00,0.0,0.0,0.99,0.0,7.88,10.84,0.00,1.48,1.48,0.0,0.49,0.00,0.00,0.00,0.00,2.46,0.49,0.49,0.00,0.49,0.00,0.00,0.49,0.00,0.00,0.49,1.97,0.49,1.48,0.00,0.00,0.0,0.49,0.49,0.99,0.49,0.49,0.00,0.0,0.99,1.97,0.49,0.99,0.0,0.00,0.49,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.49,0.49,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.49,0.0,0,0.00,0.49,0,0.49,0.00,0.00,0.0,0.00,0.49,16.75,1.48,8.37,0.49,0.00,0.00,0.99,5.42,0.00,0.00,0.0,0.00,1
248,93386,FB,206708532746370_2331895640227638,2019-04-29T10:28:37Z,MARA BIZZOTTO,F,EFDD,LEGA,Lega,governo,destra,ne,❌❌ C’è chi sta con gli IMMIGRATI CLANDESTINI e...,277,67,182,129,4739,0,problematico,Rifugiati,rifugiati,Comparativa,Neg-comp,Categoria di persone,NaN,Categoria di persone,gruppo,non 

In [116]:
unique_text=comDB.c_text.value_counts()

In [117]:
uni_counts=pd.DataFrame(unique_text)

In [118]:
pd.set_option('display.max_rows', 111)

In [119]:
u_c=uni_counts[uni_counts['c_text']<=4]

In [120]:
len(u_c[u_c['c_text']>1])

6594

In [121]:
comments['text_clean'][179]

''

In [123]:
final2

0               dire gente guerra portata casa rimpatriati
1                                       marcello perfavore
2        patria patrioti difesa proprie radici soccombe...
3                                 musulmani comandare casa
4                        odio dipende odio comandato libro
                               ...                        
78170    vediamo ruspe inviate salvini ripulire ambient...
78171    fine sente sempre richiamo africa parla diritt...
78172                  sparare ladri occhio penna barbetta
78173                              consigliatissima africa
78174    taser donne chiamare pagliacci tali vedrai eff...
Name: text_clean, Length: 75775, dtype: object

In [124]:
final2.to_csv('/home/diego/Desktop/Networks/text_cleaned.csv')